### Pydantic とは

- python 3.6 から、「型アノテーション」という機能が追加されました。
- この 型アノテーション を利用して、型ヒントの提供や、型の確認や検証を行ってくれる機能などを提供してくれるのが [pydantic](https://pydantic-docs.helpmanual.io/) です。
- 今年の pycon でも pydantic について触れるプロポーザルが採用されていました。結構アツいライブラリみたいです。


#### python の型アノテーション
- 関数定義や変数作成時に、 `: 型` を追記
- editor によっては、コードを記述したタイミングで間違いを指摘してくれる

In [1]:
name1 = "taro"
name2 : str = "taro"

In [2]:
def add(a, b):
    return a + b

def add_type(a: int, b: int) -> int:
    return a + b


In [3]:
add('1', '2')
# 実行する前から引数の型がおかしいことを指摘
add_type('1', '2')

'12'

### Pydantic 

1. `BaseModel` を継承して、pydantic モデルクラスを作成
    - `フィールド名: 型` で定義
    - 型は `typing` や pydanticが独自に定義している型を使う 
    - [typing Python 3.10.4 ドキュメント](https://docs.python.org/ja/3/library/typing.html)
    - [Field Types - pydantic](https://pydantic-docs.helpmanual.io/usage/types/#pydantic-types)
1. このクラスにデータを入れてオブジェクト化して使う


In [4]:
%load_ext blackcellmagic

In [5]:
# BaseModel を import
from pydantic import BaseModel


In [6]:
from pydantic import HttpUrl

class Cat(BaseModel): # 継承
    id : int
    message: str 
    code: int 
    filepath: HttpUrl


In [7]:
# データを入れて pydantic オブジェクトを作成
cat_1 = Cat(
    id=1,
    message="やったね",
    code=200,
    filepath="https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png",
)

# 辞書渡しでもOK。
d = {
    "id": 2,
    "message": "てへぺろ",
    "code": 404,
    "filepath": "https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png",
}
cat_2 = Cat(**d)

In [8]:
# 便利なメソッドがたくさん用意されている
print(dir(cat_1))



['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_private_attributes', '_iter', 'cod

In [9]:
# データへアクセス
cat_1.message

'やったね'

In [10]:
cat_2.dict()

{'id': 2,
 'message': 'てへぺろ',
 'code': 404,
 'filepath': HttpUrl('https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png', scheme='https', host='1.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png')}

In [11]:
cat_2.json()

'{"id": 2, "message": "\\u3066\\u3078\\u307a\\u308d", "code": 404, "filepath": "https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png"}'

In [12]:
# 型チェック。フレンドリーな例外を返してくれる
d = {
    "id": 2,
    "message": "てへぺろ",
    "code": 404,
    "filepath": "",
}
Cat(**d)

ValidationError: 1 validation error for Cat
filepath
  ensure this value has at least 1 characters (type=value_error.any_str.min_length; limit_value=1)

In [15]:
# ValidationError クラスを使うと、例外を json で取得可
from pydantic import ValidationError
try:
    Cat(**d)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "filepath"
    ],
    "msg": "ensure this value has at least 1 characters",
    "type": "value_error.any_str.min_length",
    "ctx": {
      "limit_value": 1
    }
  }
]


In [16]:
# Pydanticモデルも型として使える

from typing import List 

class StatusCode(BaseModel):
    id : int 
    code: int 
    message: str 
    # cats フィールドには Cat 型のデータをリストで持つ定義とする。デフォルト値は空リスト
    cats: List[Cat] = [] 
    
    

In [17]:
new_status_1 = {
    "id":1, 
    "code": 404,
    "message": "Not Found"
}

new_status_2 = {
    "id":2, 
    "code": 200,
    "message": "OK",
    "cats": [cat_1, cat_2]
}

status_1 = StatusCode(**new_status_1)
status_2 = StatusCode(**new_status_2)

In [18]:
status_1.dict()

{'id': 1, 'code': 404, 'message': 'Not Found', 'cats': []}

In [19]:
status_2.dict()

{'id': 2,
 'code': 200,
 'message': 'OK',
 'cats': [{'id': 1,
   'message': 'やったね',
   'code': 200,
   'filepath': HttpUrl('https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png', scheme='https', host='3.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png')},
  {'id': 2,
   'message': 'てへぺろ',
   'code': 404,
   'filepath': HttpUrl('https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png', scheme='https', host='1.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png')}]}

### 既存のモデルを継承して使う

- ベースとなるモデルを作成しを継承する
- 同じデータでも、返すデータを変えたい場合
    例： StatusCode テーブルにデータを入れたい場合、`id` は含まないデータを insert する。しかし、Selectしたデータには id が付与されている。

In [20]:
from typing import Optional

class BaseStatusCode(BaseModel):
    code: int 
    message: str 

class StatusCode(BaseStatusCode):
    id: int
    cats: List[Cat] = [] 



In [21]:
new_status_1 = {
    "id":1, 
    "code": 404,
    "message": "Not Found"
}

# このデータで db へ insert 
BaseStatusCode(**new_status_1)

BaseStatusCode(code=404, message='Not Found')

In [22]:
# db から fetch したデータはこの型へ流す
StatusCode(**new_status_1)


StatusCode(code=404, message='Not Found', id=1, cats=[])

### `orm_mode = True`

- pydantic モデルへデータを流し込むには、モデルオブジェクトを作成するか、辞書で渡すか
- sqlalchemy 等の ORM データモデルのデータオブジェクトを扱うためには <font color=red>**必ず `orm_mode = True` 設定が必要**</font>

In [26]:
# DataBase 

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import relationship


DBFILE = "sqlite:///./SQLtest.db"
engine = create_engine(DBFILE, echo=True, connect_args={"check_same_thread": False})


SessionLocal = sessionmaker(
    bind=engine,
    autocommit=False,
    autoflush=False,
)

Base = declarative_base()


class TableStatusCode(Base):
    __tablename__ = "statuscodes"
    id = Column(Integer, primary_key=True, index=True)
    code = Column(Integer, unique=True)
    message = Column(String)

    cats = relationship("TableCat", back_populates="statuscode")


class TableCat(Base):
    __tablename__ = "cats"
    id = Column(Integer, primary_key=True, index=True)
    filepath = Column(String)
    message = Column(String)
    code = Column(Integer, ForeignKey("statuscodes.code"))

    statuscode = relationship("TableStatusCode", back_populates="cats")

In [27]:
db = SessionLocal()

In [28]:
cat_from_db = db.query(TableCat).first()
cat_from_db

2022-07-24 08:47:57,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-24 08:47:57,331 INFO sqlalchemy.engine.Engine SELECT cats.id AS cats_id, cats.filepath AS cats_filepath, cats.message AS cats_message, cats.code AS cats_code 
FROM cats
 LIMIT ? OFFSET ?
2022-07-24 08:47:57,332 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (1, 0)


In [33]:
class Cat(BaseModel): 
    id : int
    message: str 
    code: int 
    filepath: HttpUrl
        
Cat(cat_from_db)


TypeError: __init__() takes exactly 1 positional argument (2 given)

In [38]:
class Cat(BaseModel): 
    id : int
    message: str 
    code: int 
    filepath: HttpUrl

    class Config:
        orm_mode = True


print(Cat.from_orm(cat_from_db))

id=1 message='やったね' code=200 filepath=HttpUrl('https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png', scheme='https', host='3.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png')


In [44]:
class StatusCode(BaseModel):
    id : int 
    code: int 
    message: str 
    # cats フィールドには Cat 型のデータをリストで持つ定義とする。デフォルト値は空リスト
    cats: List[Cat] = [] 

    class Config:
        orm_mode = True
   
status_from_db = db.query(TableStatusCode).get(2)
StatusCode.from_orm(status_from_db)

2022-07-24 09:09:36,856 INFO sqlalchemy.engine.Engine SELECT statuscodes.id AS statuscodes_id, statuscodes.code AS statuscodes_code, statuscodes.message AS statuscodes_message 
FROM statuscodes 
WHERE statuscodes.id = ?
2022-07-24 09:09:36,859 INFO sqlalchemy.engine.Engine [generated in 0.00338s] (2,)
2022-07-24 09:09:36,862 INFO sqlalchemy.engine.Engine SELECT cats.id AS cats_id, cats.filepath AS cats_filepath, cats.message AS cats_message, cats.code AS cats_code 
FROM cats 
WHERE ? = cats.code
2022-07-24 09:09:36,863 INFO sqlalchemy.engine.Engine [cached since 1115s ago] (200,)


StatusCode(id=2, code=200, message='OK', cats=[Cat(id=1, message='やったね', code=200, filepath=HttpUrl('https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png', scheme='https', host='3.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png')), Cat(id=3, message='追加の200番キャットです', code=200, filepath=HttpUrl('http://2.bp.blogspot.com/-5RqOJ4QvbXo/VEETRWSWxKI/AAAAAAAAocY/OAMZmQl4DPA/s180-c/cat_matatabi.png', scheme='http', host='2.bp.blogspot.com', tld='com', host_type='domain', port='80', path='/-5RqOJ4QvbXo/VEETRWSWxKI/AAAAAAAAocY/OAMZmQl4DPA/s180-c/cat_matatabi.png'))])

In [45]:
StatusCode.from_orm(status_from_db).cats

[Cat(id=1, message='やったね', code=200, filepath=HttpUrl('https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png', scheme='https', host='3.bp.blogspot.com', tld='com', host_type='domain', port='443', path='/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png')),
 Cat(id=3, message='追加の200番キャットです', code=200, filepath=HttpUrl('http://2.bp.blogspot.com/-5RqOJ4QvbXo/VEETRWSWxKI/AAAAAAAAocY/OAMZmQl4DPA/s180-c/cat_matatabi.png', scheme='http', host='2.bp.blogspot.com', tld='com', host_type='domain', port='80', path='/-5RqOJ4QvbXo/VEETRWSWxKI/AAAAAAAAocY/OAMZmQl4DPA/s180-c/cat_matatabi.png'))]